<a href="https://colab.research.google.com/github/Nuha4/adelaide_metrocard-cpi-fuel-correlation/blob/main/adl_metrocard_cpi_fuel_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>